In [1]:
import pandas as pd
import numpy as np

In [2]:
def clean_data(data_frame):
#     for column in data_frame:
#         if data_frame[column].count() < len(data_frame.index)/2:
#             del data_frame[column]
    
    def calc_coeff(attr_array, target, coeff):
        len_target = len(target)
        c_sum = np.sum((attr_array.dot(coeff) - target) ** 2) / (2 * len_target)
        return c_sum
    
    def grad_descent(attr_array, target, coeff, alpha, cycles):
    #coeff_list = [0] * cycles
        len_target = len(target)

        for i in range(cycles):
            # Predicted value
            pred_val = attr_array.dot(coeff)
            # Difference between prediction and Actual target
            diff = pred_val - target
            # Calculating gradient
            grad = attr_array.T.dot(diff) / len_target
            # Replace coefficient values with the new ones
            coeff = coeff - (grad * alpha)
            final_coeff = calc_coeff(attr_array, target, coeff)
            #coeff_list[i] = final_coeff
        return coeff
    df1 = data_frame[['Ht','Wt','BMI']]
    df1 = df1.dropna()
    for col in df1:
        df1[col] = np.around(df1[col], decimals = 2)
    Ht_len = len(df1['Ht'].values)
    Wt_len = len(df1['Wt'].values)
    bmi_len = len(df1['BMI'].values)
    Ht_n_ones = np.ones(Ht_len)
    Wt_n_ones = np.ones(Wt_len)
    bmi_n_ones = np.ones(bmi_len)
    bmi_attr_array = np.array([Ht_n_ones, df1['Ht'].values, df1['Wt'].values]).T
    Wt_attr_array = np.array([Ht_n_ones, df1['Ht'].values, df1['BMI'].values]).T
    Ht_attr_array = np.array([bmi_n_ones, df1['BMI'].values, df1['Wt'].values]).T
    coeff = np.array([0, 0, 0])
    bmi_target = np.array(df1['BMI'].values)
    Ht_target = np.array(df1['Ht'].values)
    Wt_target = np.array(df1['Wt'].values)
    alpha = 0.01
    
    Ht_grad_coef = grad_descent(Ht_attr_array, Ht_target, coeff, alpha, 100000)
    Wt_grad_coef = grad_descent(Wt_attr_array, Wt_target, coeff, alpha, 100000)
    bmi_grad_coef = grad_descent(Ht_attr_array, Ht_target, coeff, alpha, 100000)       
    keywords_data_frame = data_frame.columns[data_frame.columns.str.startswith('Medical_History_', 'Product_Info_')]
    keywords_data_frame = [x for x in keywords_data_frame if not x.startswith(('Medical_History_1','Medical_History_2','Medical_History_15','Medical_History_24'))]
    for i in keywords_data_frame:
        data_frame[i] = data_frame[i].fillna((data_frame[i].mode()))
    Emp_keywords_data_frame = data_frame.columns[data_frame.columns.str.startswith('Employment_Info_','InsuredInfo_')]
    Emp_keywords_data_frame = [x for x in keywords_data_frame if not x.startswith(('Employment_Info_1','Employment_Info_4','Employment_Info_6','InsuredInfo_3'))]
    for i in Emp_keywords_data_frame:
        data_frame[i] = data_frame[i].fillna((data_frame[i].mode()))
    data_frame['Insurance_History_1'] = data_frame['Insurance_History_1'].fillna(data_frame['Insurance_History_1'].mode())
    data_frame['Insurance_History_2'] = data_frame['Insurance_History_2'].fillna(data_frame['Insurance_History_2'].mode())
    data_frame['Family_Hist_1'] = data_frame['Family_Hist_1'].fillna(data_frame['Family_Hist_1'].mode())
    data_frame['Wt'] = np.around(data_frame['Wt'], decimals = 2)
    data_frame['Ht'] = np.around(data_frame['Ht'], decimals = 2)
    data_frame['BMI'] = np.around(data_frame['BMI'], decimals = 2)
    BMI_list = data_frame[(data_frame.Wt.notnull()) & (data_frame.Ht.notnull()) & (data_frame.BMI.isnull())].index.tolist()
    for i in BMI_list:
#         data_frame['BMI'].iloc[i] = (data_frame['Ht'].iloc[i] * -1.04556593) + (data_frame['Wt'].iloc[i] * 1.70351364) + 0.71074665089157529
#         data_frame['BMI'].iloc[i] = (data_frame['Ht'].iloc[i] * 1.6662764) + (data_frame['Wt'].iloc[i] * 0.07754808) + 0.23512214
#         data_frame['BMI'].iloc[i] = (data_frame['Ht'].iloc[i] * -0.8746338) + (data_frame['Wt'].iloc[i] * 1.59684629) + 0.62073836
        data_frame['BMI'].iloc[i] = (data_frame['Ht'].iloc[i] * bmi_grad_coef[1]) + (data_frame['Wt'].iloc[i] * bmi_grad_coef[2]) + bmi_grad_coef[0]
    Wt_list = data_frame[(data_frame.Ht.notnull()) & (data_frame.BMI.notnull()) & (data_frame.Wt.isnull())].index.tolist()
    for i in Wt_list:
#         data_frame['Wt'].iloc[i] = (data_frame['Ht'].iloc[i] * 0.61556981) + (data_frame['BMI'].iloc[i] * 0.57665336) - 0.41360434245416683
#         data_frame['Wt'].iloc[i] = (data_frame['Ht'].iloc[i] * 0.10120819) + (data_frame['BMI'].iloc[i] * 0.07351834) + 0.1379455
#         data_frame['Wt'].iloc[i] = (data_frame['Ht'].iloc[i] * 0.59756114) + (data_frame['BMI'].iloc[i] * 0.57567602) - 0.40025912
         data_frame['Wt'].iloc[i] = (data_frame['Ht'].iloc[i] * Wt_grad_coef[1]) + (data_frame['BMI'].iloc[i] * Wt_grad_coef[2]) + Wt_grad_coef[0]
    Ht_list = data_frame[(data_frame.Wt.notnull()) & (data_frame.BMI.notnull()) & (data_frame.Ht.isnull())].index.tolist()
    for i in Ht_list:
#         data_frame['Ht'].iloc[i] = (data_frame['Wt'].iloc[i] * 1.55752046) + (data_frame['BMI'].iloc[i] * -0.8955237) + 0.67214516469349728
#         data_frame['Ht'].iloc[i] = (data_frame['BMI'].iloc[i] * 0.18446449) + (data_frame['Wt'].iloc[i] * 0.11822116) + 0.39463004
#         data_frame['Ht'].iloc[i] = (data_frame['Wt'].iloc[i] * 1.2330216) + (data_frame['BMI'].iloc[i] * -0.67066157) + 0.66137008
        data_frame['Ht'].iloc[i] = (data_frame['BMI'].iloc[i] * Ht_grad_coef[1]) + (data_frame['BMI'].iloc[i] * Ht_grad_coef[2]) + Ht_grad_coef[0]
    data_frame['Ins_Age'] = np.around(data_frame['Ins_Age'], decimals = 2)
    data_frame['Ins_Age'] = data_frame['Ins_Age'].fillna(data_frame['Ins_Age'].mean())
#     data_frame['Ht'] = np.around(data_frame['Ht'], decimals = 2)
    data_frame['Ht'] = data_frame['Ht'].fillna(data_frame['Ht'].mean())
#     data_frame['Wt'] = np.around(data_frame['Wt'], decimals = 2)
    data_frame['Wt'] = data_frame['Wt'].fillna(data_frame['Wt'].mean())
    data_frame['Product_Info_2'] = pd.factorize(data_frame['Product_Info_2'])[0]
#     data_frame = data_frame.dropna(subset=['BMI'], how='all')
#     data_frame['BMI'] = data_frame['BMI'].fillna((-1.04556593 * data_frame['Ht']) + (1.70351364 * data_frame['Wt']) + 0.71074665089157529)
#     data_frame['BMI'] = data_frame['BMI'].fillna((data_frame['Ht'].iloc[i] * 1.6662764) + (data_frame['Wt'].iloc[i] * 0.07754808) + 0.23512214)
    data_frame['BMI'] = data_frame['BMI'].fillna((data_frame['Ht'].iloc[i] * -0.8746338) + (data_frame['Wt'].iloc[i] * 1.59684629) + 0.62073836)
    data_frame['BMI_Age'] = data_frame['BMI'] * data_frame['Ins_Age']
#     data_frame['BMI_zscore'] = (data_frame['BMI'] - data_frame['BMI'].mean())/data_frame['BMI'].std(ddof=0)
#     data_frame = data_frame.drop(['BMI'], axis = 1)
    med_keyword_columns = data_frame.columns[data_frame.columns.str.startswith('Medical_Keyword_')]
    data_frame['Med_Keywords_Count'] = data_frame[med_keyword_columns].sum(axis=1)
    data_frame['prop_BMI_Wt'] = ((data_frame['BMI'] * data_frame['Wt'])** 2)
    return data_frame

In [3]:
train = pd.read_csv("training.csv")
df_all = train
df = clean_data(df_all)
df.fillna(-1, inplace=True)
cols = list(df.columns.values)
cols.pop(cols.index('Response'))
df = df[cols+['Response']]

/Users/harshmangal/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
df.to_csv("cleaned_training.csv")

In [5]:
test_data = pd.read_csv("testing.csv")
main_solution = pd.DataFrame(test_data['Id'])
test_data = test_data.drop(['Id'], axis = 1)

In [6]:
test_data1 = clean_data(test_data)
test_data1.fillna(-1, inplace=True)

/Users/harshmangal/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
test_data1.to_csv('cleaned_testing.csv')

In [8]:
#The Attribute class which includes the column number and the value for the column number
#This class contains the value of the attribute which is used to divide the dataset at each node of the tree
class attribute:
    def __init__(self,value,index):
        self.value = value
        self.index = index      

In [9]:
#This function compares an attribute x with another attribute to see 
#which side of the tree the given attribute x should belong to
def compare_attribute(x,attribute):   
    compare_to = x[attribute.index]
    if ((type(compare_to)== type(7))or(type(compare_to)==type(7.7))):
        return (compare_to >= attribute.value)
    else:
        return (compare_to == attribute.value)

In [10]:
#This function divides the data based on the attribute
def divide_data(data,attribute):
    left_data = []
    right_data = []
    for x in data:
        if compare_attribute(x,attribute): 
            right_data.append(x)
        else:
            left_data.append(x)
    return left_data,right_data  
            

In [11]:
#Calculates gini index of given data
def gini(data):
    temp = []
    count_temp = []
    for x in data:
        if (x[len(x)-1] not in temp):
            temp.append(x[len(x)-1])
            count_temp.append(0)
        count_temp[temp.index(x[len(x)-1])] += 1
    out = 1;
    for i in range(0,len(temp)):
        curr_probability = count_temp[i]/len(data)
        out = out - curr_probability**2
    return out
            

In [12]:
#Calculates the reduction in gini index after the division has been performed
def gini_reduction(initial_gini,left_data,right_data):
    probability_left = len(left_data)/(len(left_data)+len(right_data))
    probability_right = 1-probability_left
    out = initial_gini-probability_left*gini(left_data)-probability_right*gini(right_data)
    return out

In [13]:
#Finds the best division attribute for the given data based on gini index
def division_attribute(data):
    output_attribute = None
    max_gini_reduction = 0;
    initial_gini = gini(data)
    for i in range(0,len(data[0])-1):
        column_unique_values = []
        for x in data:
            if (x[i] not in column_unique_values):
                column_unique_values.append(x[i])
        for y in column_unique_values:
            curr_attribute = attribute(y,i)
            left_data,right_data = divide_data(data,curr_attribute)
            curr_gini_reduction = gini_reduction(initial_gini,left_data,right_data)
            if ((curr_gini_reduction >= max_gini_reduction)and(curr_gini_reduction != 0)):
                max_gini_reduction = curr_gini_reduction
                output_attribute = curr_attribute
    return output_attribute
    

In [14]:
#Counts instances of all classes in the given dataset
def count_instances(data):
    out = {}
    for x in data:
        temp = x[len(x)-1]
        if temp not in out:
            out[temp] = 1
        else:
            out[temp] += 1
    return out

In [15]:
#The Node Datastructure implementation for the Decision Tree
class Node:
    def __init__(self, left_branch, right_branch, attribute, data):
        self.left_branch = left_branch
        self.right_branch = right_branch
        self.attribute = attribute
        if (attribute == None): self.content = count_instances(data) 

In [16]:
#Construction of a decision tree from given data
def make_decision_tree(data):
    if (len(data) <= 30): return Node(None,None,None,data)
    attribute = division_attribute(data)
    if (attribute == None): return Node(None,None,None,data)
    left_data, right_data = divide_data(data, attribute)
    left_branch = make_decision_tree(left_data)
    right_branch = make_decision_tree(right_data)
    return Node(left_branch, right_branch, attribute,data)
    

In [17]:
#Predicts the class of sample from the tree with root node
def predict(sample, node):
    if (node.attribute == None):
        return node.content
    if compare_attribute(sample,node.attribute):
        return predict(sample, node.right_branch)
    else:
        return predict(sample, node.left_branch)

In [18]:
#Makes a dataset based on the weights
import random
def make_data_set(training_data,weights):
    curr_set = []
    for i in range(0,len(training_data)):
        random_number  = random.uniform(0,1)
        sum = 1/len(training_data)
        for j in range(0,len(weights)):
            if (random_number < sum):
                curr_set.append(training_data[j])
                break
            sum +=1/len(training_data)    
    return curr_set

In [19]:
#Merges the prediction results of multiple trees
def merge(dicts):
    out = {}
    for data in dicts:
        for x in data.keys():
            if x not in out:
                out[x] = 0
            out[x] += data[x] 
    return out

In [20]:
#Returns a class based on multiple prediction dictionaries
def predict_result(dict_predictions):    
    curr_dict = dict_predictions
    out = 0 
    num = 0
    for x in curr_dict.keys():
        if (curr_dict[x]>num):
            out = int(x)
            num = curr_dict[x]
    return out

In [21]:
#Assigns votes to the classifier results
def process(dict,vote):
    out  = {}
    for x in dict.keys():
        out[x]= dict[x]*vote
    return out

In [22]:
# def get_label_weights(training_data):
#     a = count_instances(training_data)
#     for x in a.keys():
#         a[x] = len(training_data)/a[x]
#     return a


In [23]:
# def multiply_dicts(a,b):
#     out = {}
#     for x in a.keys():
#         out[x] = a[x]*b[x]
#     return out    

In [24]:
#Calculates gini index of the predictions results dictionary
def gini_dict(dict_):
    temp = dict_
    sum_temp = 0
    for x in temp.keys():
        sum_temp+=temp[x]
    out  = 1
    for x in temp.keys():
        temp[x] = temp[x]/sum_temp
        out = out - temp[x]**2
    return out

In [25]:
#Returns the average of all classes in the given dataset
def predict_average(dict_predictions):    
    curr_dict = dict_predictions
    out = 0 
    num = 0            
    for x in curr_dict.keys():
            out = out+x*curr_dict[x]
            num = num+curr_dict[x]
    out = int(round((out/num-1)*8/7,1))
    if (out>8): out = 8
    if (out<1): out = 1
    return out

In [26]:
# def check_and_predict_average(dict_predictions):    
#     curr_dict = dict_predictions
#     out = 0 
#     num = 0
#     for x in curr_dict.keys():
#         c = 0
#         for y in curr_dict.keys():
#             if((x!=y)and(curr_dict[x]<2*curr_dict[y])): c = 1
#         if (c==0): return int(x)        
                
#     for x in curr_dict.keys():
#             out = out+x*curr_dict[x]
#             num = num+curr_dict[x]
#     #print("Out = %s and Num = %s" %(out,num))        
#     out = int(round((out/num-1)*8/7,1))
#     if (out>8): out = 8
#     if (out<1): out = 1
#     return out

In [27]:
##Reading from the file
fo = open("cleaned_training.csv", "r")

a = fo.readline()
a = a.strip().split(',')
a = a[2:]
Column_names = a
training_data = []
for i in range(0,20000):
    temp = fo.readline()
    temp = temp.strip().split(',')
    temp = temp[2:]
    temp = [round(float(a),1) for a in temp]
    training_data.append(temp)    

In [28]:
##Assigning Weights for AdaBoost
weights = [1/len(training_data)]*len(training_data)

In [29]:
#making 10 trees using AdaBoosting
import math
trees = [] #The trees arrays
votes = [] #The votes of each tree array

num_trees = 100

for j in range(0,num_trees):
    print("Starting Classfier number: "+str(j))
    curr_set = make_data_set(training_data,weights) #Get the dataset based on weights
    curr_tree = make_decision_tree(curr_set) #Make decision tree from the dataset
    trees.append(curr_tree)
    print("Tree Constrcuted")
    error = 0
    correctness_array = []
    for i in range(0,len(training_data)): #Checking predictions for the training set to calculate error
        
        d = []
        for k in range(0,j+1):
            temp = predict(training_data[i],trees[k])
            d.append(temp)
        oo = merge(d)
        if (gini_dict(oo)<0.4):
             x= predict_result(oo)
        else:
            x = predict_average(oo) #x is the predicted class for the current tuple
        #temp = predict(row,curr_tree) 
        actual = int(training_data[i][-1])
        if (actual!=x):
            error = error+ weights[i]*((abs(actual-x)/7)**2)
            correctness_array.append(-(abs(actual-x)/7)**2)
        else:
            correctness_array.append(1)
    print("Error= "+str(error))        
    curr_vote = 0.5*math.log((1-error)/error) ##Calculating vote for current classifier
    votes.append(curr_vote)
    print("Vote= "+str(curr_vote))
    for i in range(0,len(weights)):
        weights[i] = weights[i]*math.exp(-curr_vote*correctness_array[i]) ##Updating weights
    sum_weights = sum(weights)
    print("Sum of weights = " + str(sum_weights))
    weights = [a/sum_weights for a in weights] ##Normalizing weights       

Starting Classfier number: 0
Tree Constrcuted
Error= 0.08667346938775337
Vote= 1.1774728151244385
Sum of weights = 0.807413266534544
Starting Classfier number: 1
Tree Constrcuted
Error= 0.12481582322314887
Vote= 0.9737975577038988
Sum of weights = 0.9598207912058743
Starting Classfier number: 2
Tree Constrcuted
Error= 0.17406871281750372
Vote= 0.7785307300593732
Sum of weights = 1.0650226199945867
Starting Classfier number: 3
Tree Constrcuted
Error= 0.22921591591838303
Vote= 0.606371932073437
Sum of weights = 1.1121937468583845
Starting Classfier number: 4
Tree Constrcuted
Error= 0.2821567527468852
Vote= 0.4668942246574976
Sum of weights = 1.1230695783517335
Starting Classfier number: 5
Tree Constrcuted
Error= 0.32561854040638305
Vote= 0.3640346699083365
Sum of weights = 1.115929353668898
Starting Classfier number: 6
Tree Constrcuted
Error= 0.3638404023743934
Vote= 0.2793670775255514
Sum of weights = 1.1004581577058221
Starting Classfier number: 7
Tree Constrcuted
Error= 0.396830955937

Error= 0.5009657455425891
Vote= -0.0019314934870943718
Sum of weights = 0.9990863437966525
Starting Classfier number: 61
Tree Constrcuted
Error= 0.4969476157148107
Vote= 0.006104844409984978
Sum of weights = 1.0028715761646696
Starting Classfier number: 62
Tree Constrcuted
Error= 0.4978357072427334
Vote= 0.004328612549238039
Sum of weights = 1.002039138800485
Starting Classfier number: 63
Tree Constrcuted
Error= 0.4978794726075904
Vote= 0.004241080212402229
Sum of weights = 1.00199878784518
Starting Classfier number: 64
Tree Constrcuted
Error= 0.4988405985510948
Vote= 0.002318807053773122
Sum of weights = 1.0010946611929368
Starting Classfier number: 65
Tree Constrcuted
Error= 0.4979681852352859
Vote= 0.004063651897336332
Sum of weights = 1.001918014782327
Starting Classfier number: 66
Tree Constrcuted
Error= 0.4980799061614328
Vote= 0.003840206554437036
Sum of weights = 1.0018115818281865
Starting Classfier number: 67
Tree Constrcuted
Error= 0.4979802480135867
Vote= 0.0040395259447014

In [30]:
# #checking the accuracy for training set(first 100 tuples)
# #label_weights = get_label_weights(training_data)
# for row in training_data[:100]:
#     d = []
#     for i in range(0,num_trees):
#         temp = predict(row,trees[i])
#         #temp = process(temp,votes[i])
#         d.append(temp)
#     oo = (merge(d))
#     #oo = multiply_dicts(oo,label_weights)
#     ll = predict_average(oo)
#     print(gini_dict(oo))
#     if (gini_dict(oo)<0.4):
#         ll = predict_result(oo)
#     else:
#         ll = predict_average(oo)
#     print ("Actual: %s. Predicted: %s and result: %s" %
#            (row[-1], oo,ll))

In [31]:
#Reading the test dataset
f2 = open("cleaned_testing.csv","r")
testing_data = []
header = f2.readline()
for i in range(0,10000):
    temp = f2.readline()
    temp = temp.strip().split(',')
    temp = temp[1:]
    temp = [round(float(a),1) for a in temp]
    testing_data.append(temp) 
len(testing_data)


10000

In [32]:
#label_weights = get_label_weights(training_data)
dict_predictions = []
for row in testing_data:
    d = []
    
    for i in range(0,num_trees):
        temp = predict(row,trees[i])
        #temp = process(temp,votes[i])
        d.append(temp)
    oo = merge(d)
    #oo = multiply_dicts(oo,label_weights)
    if (gini_dict(oo)<0.4):
        ll = predict_result(oo)
    else:
        ll = predict_average(oo)
    dict_predictions.append(ll)

In [33]:
dict_predictions

[8,
 8,
 8,
 6,
 8,
 7,
 5,
 4,
 5,
 7,
 6,
 6,
 8,
 5,
 7,
 2,
 6,
 7,
 4,
 8,
 6,
 6,
 8,
 8,
 5,
 4,
 8,
 8,
 8,
 7,
 8,
 8,
 8,
 4,
 4,
 6,
 6,
 8,
 7,
 4,
 5,
 8,
 6,
 3,
 4,
 2,
 6,
 8,
 8,
 6,
 5,
 5,
 8,
 5,
 3,
 4,
 8,
 3,
 5,
 5,
 8,
 8,
 3,
 7,
 5,
 8,
 6,
 5,
 8,
 4,
 8,
 3,
 8,
 5,
 4,
 8,
 3,
 4,
 8,
 8,
 8,
 8,
 6,
 8,
 6,
 6,
 5,
 3,
 4,
 5,
 6,
 4,
 6,
 5,
 6,
 7,
 8,
 8,
 8,
 7,
 7,
 5,
 8,
 1,
 5,
 8,
 4,
 8,
 4,
 6,
 8,
 4,
 7,
 4,
 6,
 5,
 4,
 3,
 8,
 3,
 7,
 2,
 8,
 8,
 3,
 4,
 6,
 8,
 8,
 7,
 7,
 4,
 2,
 6,
 8,
 8,
 8,
 6,
 4,
 2,
 5,
 2,
 5,
 5,
 4,
 8,
 8,
 1,
 5,
 8,
 1,
 8,
 4,
 3,
 1,
 6,
 8,
 8,
 5,
 1,
 5,
 8,
 8,
 3,
 8,
 4,
 6,
 8,
 6,
 8,
 5,
 5,
 7,
 4,
 8,
 5,
 8,
 2,
 3,
 5,
 6,
 5,
 8,
 8,
 2,
 5,
 5,
 5,
 4,
 6,
 3,
 2,
 8,
 2,
 5,
 7,
 6,
 5,
 7,
 7,
 6,
 6,
 8,
 6,
 8,
 6,
 8,
 8,
 5,
 6,
 6,
 4,
 8,
 6,
 6,
 6,
 1,
 8,
 4,
 8,
 4,
 6,
 2,
 3,
 1,
 4,
 6,
 1,
 8,
 8,
 1,
 8,
 4,
 5,
 6,
 6,
 4,
 8,
 6,
 5,
 8,
 6,
 6,
 2,
 6,
 6,
 5,
 8,
 5,
 5,


In [34]:
#Writing the results        
f2 = open("out_boosting_10_classifiers_gini_improved_and_updated_average_with_votes.csv","w")
f2.write("Id,Response\n")
for i in range(0,10000):
    hh = str(20000+i)+","+str(dict_predictions[i])+"\n"
    f2.write(hh) 

f2.close()